In [1]:
import os
from web3 import Web3
from dotenv import load_dotenv
from web3.middleware import geth_poa_middleware
from eth_account import Account

In [2]:
from pathlib import Path

In [3]:
load_dotenv()

True

In [4]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [5]:
account_one = Account.from_key(os.getenv("PRIVATE_KEY"))

In [6]:
from getpass import getpass
ks_fname = "UTC--2021-06-27T20-30-54.023984200Z--8849805797df36d0b7d71057c89f1217b18e5f05"
with open(Path("../../Blockchain-Tools/node2/keystore/" + ks_fname)) as keyfile:
    encrypted_key = keyfile.read()
    print(encrypted_key)
    private_key = w3.eth.account.decrypt(
        encrypted_key, getpass("Enter keystore password: ")
    )
    account_two = Account.from_key(private_key)

{"address":"8849805797df36d0b7d71057c89f1217b18e5f05","crypto":{"cipher":"aes-128-ctr","ciphertext":"a2c21804551bd33fac65fad418382d9accfa61123ae98f9b7140e13e5b23746c","cipherparams":{"iv":"30be3cef9ddf49651e51a56611dd86f2"},"kdf":"scrypt","kdfparams":{"dklen":32,"n":262144,"p":1,"r":8,"salt":"d05ed0716060de8f21f657b09f2fd5df02ac6c19af54f63be617377cda0dabb9"},"mac":"894204e25916ac8b932767614beee9a7b4e3959fc644e0ff1a8c495edefb770f"},"id":"465332a7-6836-42e5-93c1-3b8d3c0a8e22","version":3}


Enter keystore password:  ····


In [7]:
account_two.address

'0x8849805797df36d0B7D71057c89F1217B18e5f05'

In [8]:
def create_raw_tx(account, recipient, amount):
    gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": recipient, "value": amount}
    )
    return {
        "from": account.address,
        "to": recipient,
        "value": amount,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address),
    }

In [9]:
def send_tx(account, recipient, amount):
    tx = create_raw_tx(account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    print(result.hex())
    return result.hex()

In [10]:
print(account_one.address)
print(account_two.address)

0x18C91C1fc3A57C7961a510D23b454161317c73c4
0x8849805797df36d0B7D71057c89F1217B18e5f05


In [12]:
send_tx(account_two, account_one.address, 1)

0x7c0b857a3dc840237b67ac8db6ebcfa5bf5a39e6def3e2bf071c3f7c87d335e5


'0x7c0b857a3dc840237b67ac8db6ebcfa5bf5a39e6def3e2bf071c3f7c87d335e5'